In [ ]:
import pandas as pd
#ground = find the ground thruth from file/annotations_external_dataset.csv
ground = pd.read_csv('annotations_external_dataset.csv')

In [ ]:
# in the column filename i leave PatientXXX. 3 patient are missing, total patient number 283, but real 280.
ground["filename"] = ground["filename"].str.extract(r"(Patient\d{3})")

In [ ]:
#'non gradable' and missing values become 0
ground['labels'] = (ground['labels'].replace("non gradable", 0).fillna(0).astype(int))

# add of a new column x, which is the binary outcome 0,1
ground["x"] = (ground.groupby("filename")["labels"].transform(lambda s: s.astype(str).isin(["1","2","3"]).any()).astype(int))


In [ ]:
#count of how many 0,1 in x there are per patient
counts = (ground.groupby("filename")["x"].first().value_counts())
print(counts)

In [ ]:
#i want to add the new column x to the file external_merged_LE_RE.csv, which i now call merg
merg = pd.read_csv('/external_merged_LE_RE.csv')

In [ ]:
# i extract only the patient number from the mask_name
merg["mask_name"] = merg["mask_name"].str.extract(r"(Patient\d{3})")
#rename of the column aas mask_name to allow for the join
merg = merg.rename(columns={"mask_name": "filename"})

In [ ]:
# left join 
x_by_file = ground.groupby("filename", as_index=False)["x"].first()
merg = merg.merge(x_by_file, on="filename", how="left")

In [ ]:
merg['outcome']

In [ ]:
#drop the column outcome
merg = merg.drop(columns=["outcome"])
#rename of the column x as outcome and file name as mask_name (this allows to run the code after)
merg = merg.rename(columns={"x": "outcome"})
merg = merg.rename(columns={"filename": "mask_name"})
#insert your pathname
merg.to_csv("external_test.csv", index=False)